In this folder, merged_file.csv is the data set from all NHTSA files, and NHTSA_ADS_cleaned.csv is the cleaned data from ../california_data/NHTSA _SGO_incident_reports/SGO-2021-01_Incident_Reports_ADS.csv
1. delete if "NHTSA_ADS_cleaned.csv" is existed
2. read ../california_data/NHTSA _SGO_incident_reports/SGO-2021-01_Incident_Reports_ADS.csv

## Steps to processing data (columns)
1. Report ID -- no change
2. Report Version -- no change
3. Reporting Entity -- no change
4. Report Type -- one hot encoded
5. Report Month -- no change
6. Report Year -- no change
7. Report Submission Date -- break down to year and month
8. VIN -- no change
9. VIN - Unknown -- one hot encoded
10. Serial Number -- dropped, no information provided
11. Make, model -- no change
12. Model -- Unknown - there's only 1 'Y' in this column, so I dropped the row with 'Y', and dropped 'Model - Unknown' col
13. Model Year -- no change
14. Model Year -- Unknown - there's only 4 'Y', so same as above
15. Same Vehicle ID, Mileage -- no change
16. Mileage - Unknown -- one hot encoded
17. Driver / Operator Type -- one hot encoded, with "None", "Other, see Narrative" and "Unknown" combined into the same category, also put 1 on both "In-Vehicle" col and "Remote" col instead of have a category for "In-Vehicle and Remote (Commercial / Test)"
18. ADAS/ADS System Version, ADAS/ADS Hardware Version, ADAS/ADS Software Version -- no change
19. ADAS/ADS System Version - Unk, ADAS/ADS System Version CBI, ADAS/ADS Hardware Version - Unk, ADAS/ADS Hardware Version CBI, ADAS/ADS Software Version - Unk, ADAS/ADS Software Version CBI -- one hot encoded
20. Other Reporting Entities? -- no change
21. Other Reporting Entities? -- Unk, Other Reporting Entities? - NA - one hot encoded
22. Federal Regulatory Exemption? -- no change(less information provided, seems not important?)
23. Other Federal Reg. Exemption -- dropped, no info
24. Federal Reg. Exemption - Unk, Federal Reg. Exemption - No, State or Local Permit? -- one hot encoded, for 'State or Local Permit?' I put 0 for unknown as well
25. State or Local Permit -- no change
26. ADS Equipped?, Automation System Engaged? -- one hot encoded
27. Operating Entity -- no change
28. Operating Entity - Unknown -- dropped, no info
29. Source - Complaint/Claim,Source - Telematics,Source - Law Enforcement,Source - Field Report,Source - Testing,Source - Media,Source - Other -- one hot encoded
30. Source - Other Text -- no change
31. Incident Date, Notice Received Date -- break down into year and month
32. Incident Date - Unknown, Notice Received Date - Unknown -- one hot encoded
33. Incident Time (24:00), Same Incident ID -- no change
34. Latitude, Latitude - Unknown, Longitude, Longitude - Unknown, Address, Address - Unknown, Zip Code, Zip Code - Unknown -- dropped, no info
35. City, State -- no change
36. City - Unknown -- one hot encoded
37. Roadway Type, Roadway Surface, Roadway Description -- one hot encoded
38. Posted Speed Limit -- no change
39. Posted Speed Limit -- Unknown - one hot encoded
40. Lighting, Weather - Clear, Weather - Snow, Weather - Cloudy, Weather - Fog/Smoke, Weather - Rain, Weather - Severe Wind, Weather - Unknown, Weather - Other -- one hot encoded ('Weather - Unknown' and 'Weather - other' are cols that provided on the original data, maybe we should combine them into the same category?)
41. Weather - Other Text -- no change
42. Crash With -- no change, this col indicate what type of car did the vehicle crash with, seems reasonable ot encoded this?
43. Highest Injury Severity Alleged, Property Damage? -- one hot encoded
44. CP Pre-Crash Movement -- no change
45. CP Any Air Bags Deployed?, CP Was Vehicle Towed?, CP Contact Area - Rear Left, CP Contact Area - Left, CP Contact Area - Front Left, CP Contact Area - Rear, CP Contact Area - Top, CP Contact Area - Front, CP Contact Area - Rear Right, CP Contact Area - Right, CP Contact Area - Front Right, CP Contact Area - Bottom, CP Contact Area - Unknown -- one hot encoded
46. SV Pre-Crash Movement -- no change
47. SV Any Air Bags Deployed?, SV Was Vehicle Towed?, SV Were All Passengers Belted? -- one hot encoded
48. SV Precrash Speed (MPH) -- no change
49. SV Pre-crash Speed - Unknown, SV Contact Area - Rear Left, SV Contact Area - Left, SV Contact Area - Front Left, SV Contact Area - Rear, SV Contact Area - Top, SV Contact Area - Front, SV Contact Area - Rear Right, SV Contact Area - Right, SV Contact Area - Front Right, SV Contact Area - Bottom, SV Contact Area - Unknown, Data Availability - EDR, Data Availability - Police Rpt, Data Availability - Telematics, Data Availability - Complaints, Data Availability - Video, Data Availability - Other, Data Availability - Unknown -- one hot encoded
50. Data Availability - No Data -- dropped, no info

*for all one hot encoded cols after 'CP Any Air Bags Deployed?', 1 = Yes or 'Y', 0 = 'N', 'No','Unknown', or 'Not Applicable'
*original cols are dropped after one hot encoded if the col contains multiple types

cols after 'Law Enforcement Investigating?' have not been processed!


In [1577]:
one_hot_encoded = pd.get_dummies(merged_df['Report Type'], prefix='Report_Type')

desired_order = [
    'Report_Type_1 Day Update', 
    'Report_Type_5 Day Update', 
    'Report_Type_10 Day Update', 
    'Report_Type_Monthly Report', 
    'Report_Type_Updated, but no time indicated', 
]

one_hot_encoded = one_hot_encoded.reindex(columns=desired_order)

for col in reversed(one_hot_encoded.columns):
    merged_df.insert(merged_df.columns.get_loc('Report Type') + 1, col, one_hot_encoded[col])

merged_df.drop(columns= 'Report Type', inplace= True)

In [1578]:
import pandas as pd
import numpy as np

import os

file_path = "NHTSA_ADS_cleaned.csv"
if os.path.exists(file_path):
    os.remove(file_path)
    print(f"{file_path} has been deleted.")
else:
    print(f"{file_path} does not exist.")

NHTSA_ADS_cleaned.csv has been deleted.


### Merge all NHTSA file into one csv file

In [1579]:
# file_names = ["../california_data/NHTSA _SGO_incident_reports/SGO-2021-01_Incident_Reports_ADAS_March2024.csv", 
#               "../california_data/NHTSA _SGO_incident_reports/SGO-2021-01_Incident_Reports_ADAS.csv", 
#               "../california_data/NHTSA _SGO_incident_reports/SGO-2021-01_Incident_Reports_ADS.csv", 
#               "../california_data/NHTSA _SGO_incident_reports/SGO-2021-01_Incident_Reports_March2024.csv", 
#               "../california_data/NHTSA _SGO_incident_reports/SGO-2021-01_Incident_Reports_OTHER.csv"]

# dataframes = []
# for file in file_names:
#     df = pd.read_csv(file)
#     dataframes.append(df)

# merged_df = pd.concat(dataframes, ignore_index=True)
# merged_df.to_csv("merged_file.csv", index=False)

In [1580]:
file_names = ["../california_data/NHTSA _SGO_incident_reports/SGO-2021-01_Incident_Reports_ADS.csv"]

dataframes = []
for file in file_names:
    df = pd.read_csv(file)
    dataframes.append(df)

merged_df = pd.concat(dataframes, ignore_index=True)

### Processing columns of merged data
check entity count

In [1581]:
unique_entities = merged_df['Reporting Entity'].nunique()
print("Reporting Entity count: ", unique_entities)
# pd.set_option('display.max_rows', None)
print(merged_df['Reporting Entity'].value_counts())


Reporting Entity count:  34
Waymo LLC                                   196
Cruise LLC                                  126
General Motors, LLC                         119
Transdev Alternative Services                97
Zoox, Inc.                                   39
Argo AI                                      20
Ford Motor Company                           17
May Mobility                                 13
Toyota Motor Engineering & Manufacturing      7
Pony.ai                                       6
Mercedes-Benz USA, LLC                        6
Easymile Inc.                                 6
WeRide Corp                                   6
Kodiak Robotics                               4
Lucid USA, Inc.                               4
Motional                                      4
Beep, Inc.                                    4
PACCAR Incorporated                           3
Navistar, Inc.                                3
Aurora Operations, Inc.                       3
Hyundai Moto

It is hard and it seems like meaningless to encode the entities' name
<br>if you want to drop the "Reporting Entity" col, use: merged_df.drop(columns= 'Reporting Entity', inplace= True)

In [1582]:
merged_df['Report Type'] = merged_df['Report Type'].replace({
    "1-Day": "1 Day Update",
    "5-Day": "5 Day Update",
    "10-Day Update": "10 Day Update",
    "Update": "Updated, but no time indicated",
    "Monthly": "Monthly Report",
    "No New or Updated Incident Reports": "Updated, but no time indicated"
})
print(merged_df['Report Type'].value_counts())

Monthly Report                    408
10 Day Update                     144
5 Day Update                       58
Updated, but no time indicated     57
1 Day Update                       36
Name: Report Type, dtype: int64


Using One-hot to encode report type, then drop "report type col"

In [1583]:
one_hot_encoded = pd.get_dummies(merged_df['Report Type'], prefix='Report_Type')

desired_order = [
    'Report_Type_1 Day Update', 
    'Report_Type_5 Day Update', 
    'Report_Type_10 Day Update', 
    'Report_Type_Monthly Report', 
    'Report_Type_Updated, but no time indicated', 
]

one_hot_encoded = one_hot_encoded.reindex(columns=desired_order)

for col in reversed(one_hot_encoded.columns):
    merged_df.insert(merged_df.columns.get_loc('Report Type') + 1, col, one_hot_encoded[col])

merged_df.drop(columns= 'Report Type', inplace= True)

In [1584]:
print(merged_df['Report Year'].value_counts().sum())
print(merged_df['Report Month'].value_counts().sum())
# merged_df.drop(columns= 'Report Year', inplace= True)
# merged_df.drop(columns= 'Report Month', inplace= True)

430
430


Break submission date into year and month, drop the original col

In [1585]:
merged_df['Report Submission Date'] = pd.to_datetime(merged_df['Report Submission Date'], format='%b-%Y', errors='coerce')
merged_df.insert(merged_df.columns.get_loc('Report Submission Date') + 1, 'Submission Year', merged_df['Report Submission Date'].dt.year)
merged_df.insert(merged_df.columns.get_loc('Report Submission Date') + 2, 'Submission Month', merged_df['Report Submission Date'].dt.month)
merged_df.drop(columns= 'Report Submission Date', inplace= True)


In [1586]:
# cols below seems less important
# merged_df.drop(columns= 'VIN', inplace= True)
# merged_df.drop(columns= 'VIN - Unknown', inplace= True)

merged_df['VIN - Unknown'] = merged_df['VIN - Unknown'].apply(lambda x: 1 if x == 'Y' else 0)

merged_df.drop(columns= 'Serial Number', inplace= True)


Not sure what should I do for 'make' col and 'Model' col, one-hot encoding seems like reasonable?
<br>
Drop row if 'Model - Unknown' col is 'Y', then drop the 'Model - Unknown' col
<br>
do the same thing for 'Model Year - Unknown' col and 'milage' col

In [1587]:
print(merged_df['Make'].nunique())
print(merged_df['Model'].nunique())
merged_df = merged_df[merged_df['Model - Unknown'] != 'Y']
merged_df.drop(columns= 'Model - Unknown', inplace= True)
merged_df = merged_df[merged_df['Model Year - Unknown'] != 'Y']
merged_df.drop(columns= 'Model Year - Unknown', inplace= True)
merged_df['Mileage - Unknown'] = merged_df['Mileage - Unknown'].apply(lambda x: 1 if x == 'Y' else 0)



31
55


using one-hot to encode this col, (combine other and none, if the operator type is 'In-Vehicle and Remote', put 1 on both 'In-Vehicle' and 'Remote', instead of have a unique col)

In [1588]:
merged_df['Driver / Operator Type'] = merged_df['Driver / Operator Type'].replace({
    "In-Vehicle (Commercial / Test)": "In-Vehicle",
    "Remote (Commercial / Test)": "Remote",
    "None": "None or Other",
    "Other, see Narrative": "None or Other",
    "Unknown": "None or Other"
})
# print(merged_df['Driver / Operator Type'].value_counts())

one_hot_encoded = pd.get_dummies(merged_df['Driver / Operator Type'], prefix='Driver / Operator Type')

if 'Driver / Operator Type_In-Vehicle and Remote (Commercial / Test)' in one_hot_encoded.columns:
    one_hot_encoded['Driver / Operator Type_In-Vehicle'] |= one_hot_encoded['Driver / Operator Type_In-Vehicle and Remote (Commercial / Test)']
    one_hot_encoded['Driver / Operator Type_Remote'] |= one_hot_encoded['Driver / Operator Type_In-Vehicle and Remote (Commercial / Test)']
    one_hot_encoded = one_hot_encoded.drop(columns=['Driver / Operator Type_In-Vehicle and Remote (Commercial / Test)'])

desired_order = [
    'Driver / Operator Type_Consumer', 
    'Driver / Operator Type_In-Vehicle', 
    'Driver / Operator Type_Remote', 
    'Driver / Operator Type_None or Other', 
]

one_hot_encoded = one_hot_encoded.reindex(columns=desired_order)

for col in reversed(one_hot_encoded.columns):
    merged_df.insert(merged_df.columns.get_loc('Driver / Operator Type') + 1, col, one_hot_encoded[col])

merged_df.drop(columns= 'Driver / Operator Type', inplace= True)

process all data with UNK and CBI

In [1589]:
print(merged_df['ADAS/ADS System Version'].value_counts())
# print(merged_df['ADAS/ADS System Version CBI'].value_counts())
merged_df['ADAS/ADS System Version - Unk'] = merged_df['ADAS/ADS System Version - Unk'].apply(lambda x: 1 if x == 'Y' else 0)
merged_df['ADAS/ADS System Version CBI'] = merged_df['ADAS/ADS System Version CBI'].apply(lambda x: 1 if x == 'Y' else 0)
print(merged_df['ADAS/ADS Hardware Version'].nunique())
merged_df['ADAS/ADS Hardware Version - Unk'] = merged_df['ADAS/ADS Hardware Version - Unk'].apply(lambda x: 1 if x == 'Y' else 0)
merged_df['ADAS/ADS Hardware Version CBI'] = merged_df['ADAS/ADS Hardware Version CBI'].apply(lambda x: 1 if x == 'Y' else 0)
print(merged_df['ADAS/ADS Software Version'].nunique())
merged_df['ADAS/ADS Software Version - Unk'] = merged_df['ADAS/ADS Software Version - Unk'].apply(lambda x: 1 if x == 'Y' else 0)
merged_df['ADAS/ADS Software Version CBI'] = merged_df['ADAS/ADS Software Version CBI'].apply(lambda x: 1 if x == 'Y' else 0)
print(merged_df['Other Reporting Entities?'].nunique())
merged_df['Other Reporting Entities? - Unk'] = merged_df['Other Reporting Entities? - Unk'].apply(lambda x: 1 if x == 'Y' else 0)
merged_df['Other Reporting Entities? - NA'] = merged_df['Other Reporting Entities? - NA'].apply(lambda x: 1 if x == 'Y' else 0)


5th Generation ADS                                           211
[REDACTED, MAY CONTAIN CONFIDENTIAL BUSINESS INFORMATION]    192
5th Generation                                                30
4                                                             20
4th Generation ADS                                            15
L4                                                             6
Voyager 11 (21.02.4)                                           3
Level 2                                                        2
DRIVE PILOT                                                    2
Level 4                                                        2
Test Version 04062022                                          2
53.22.10.03                                                    1
Pilot Assist                                                   1
95.22.04.22                                                    1
36.22.30.08                                                    1
04.22.09.01              

Dropping 'Other Federal Reg. Exemption' col since it's completely blank

In [1590]:
merged_df.drop(columns= 'Other Federal Reg. Exemption', inplace= True)

process data:

In [1591]:
merged_df['Federal Reg. Exemption - Unk'] = merged_df['Federal Reg. Exemption - Unk'].apply(lambda x: 1 if x == 'Y' else 0)
merged_df['Federal Reg. Exemption - No'] = merged_df['Federal Reg. Exemption - No'].apply(lambda x: 1 if x == 'Y' else 0)
merged_df['State or Local Permit?'] = merged_df['State or Local Permit?'].apply(lambda x: 1 if x == 'Yes (specify below)' else 0)
merged_df['ADS Equipped?'] = merged_df['ADS Equipped?'].apply(lambda x: 1 if x == 'Yes' else 0)


One-Hot on 'Automation System Engaged?' col

In [1592]:
one_hot_encoded = pd.get_dummies(merged_df['Automation System Engaged?'], prefix='Automation System Engaged')

# Insert the one-hot encoded columns after the original 'Automation System Engaged?' column
for col in reversed(one_hot_encoded.columns):
    merged_df.insert(merged_df.columns.get_loc('Automation System Engaged?') + 1, col, one_hot_encoded[col])

merged_df.drop(columns= 'Automation System Engaged?', inplace= True)

processing data:

In [1593]:
merged_df.drop(columns= 'Operating Entity - Unknown', inplace= True)
merged_df['Source - Complaint/Claim'] = merged_df['Source - Complaint/Claim'].apply(lambda x: 1 if x == 'Y' else 0)
merged_df['Source - Telematics'] = merged_df['Source - Telematics'].apply(lambda x: 1 if x == 'Y' else 0)
merged_df['Source - Law Enforcement'] = merged_df['Source - Law Enforcement'].apply(lambda x: 1 if x == 'Y' else 0)
merged_df['Source - Field Report'] = merged_df['Source - Field Report'].apply(lambda x: 1 if x == 'Y' else 0)
merged_df['Source - Testing'] = merged_df['Source - Testing'].apply(lambda x: 1 if x == 'Y' else 0)
merged_df['Source - Media'] = merged_df['Source - Media'].apply(lambda x: 1 if x == 'Y' else 0)
merged_df['Source - Other'] = merged_df['Source - Other'].apply(lambda x: 1 if x == 'Y' else 0)
merged_df['Incident Date - Unknown'] = merged_df['Incident Date - Unknown'].apply(lambda x: 1 if x == 'Y' else 0)
merged_df['Incident Time - Unknown'] = merged_df['Incident Time - Unknown'].apply(lambda x: 1 if x == 'Y' else 0)

In [1594]:
merged_df['Incident Date'] = pd.to_datetime(merged_df['Incident Date'], format='%b-%Y', errors='coerce')
merged_df.insert(merged_df.columns.get_loc('Incident Date') + 1, 'Incident Year', merged_df['Incident Date'].dt.year)
merged_df.insert(merged_df.columns.get_loc('Incident Date') + 2, 'Incident Month', merged_df['Incident Date'].dt.month)
merged_df.drop(columns= 'Incident Date', inplace= True)

merged_df['Notice Received Date'] = pd.to_datetime(merged_df['Notice Received Date'], format='%b-%Y', errors='coerce')
merged_df.insert(merged_df.columns.get_loc('Notice Received Date') + 1, 'Notice Received Year', merged_df['Notice Received Date'].dt.year)
merged_df.insert(merged_df.columns.get_loc('Notice Received Date') + 2, 'Notice Received Month', merged_df['Notice Received Date'].dt.month)
merged_df.drop(columns= 'Notice Received Date', inplace= True)


Dropping latitude, longitude and address, zip code, because they are "[MAY CONTAIN PERSONALLY IDENTIFIABLE INFORMATION]"

In [1595]:
merged_df.drop(columns= 'Latitude', inplace= True)
merged_df.drop(columns= 'Latitude - Unknown', inplace= True)
merged_df.drop(columns= 'Longitude', inplace= True)
merged_df.drop(columns= 'Longitude - Unknown', inplace= True)
merged_df.drop(columns= 'Address', inplace= True)
merged_df.drop(columns= 'Address - Unknown', inplace= True)
merged_df.drop(columns= 'Zip Code', inplace= True)
merged_df.drop(columns= 'Zip Code - Unknown', inplace= True)

merged_df['City - Unknown'] = merged_df['City - Unknown'].apply(lambda x: 1 if x == 'Y' else 0)

In [1596]:
one_hot_encoded = pd.get_dummies(merged_df['Roadway Type'], prefix='Roadway Type')

# put 'unknown' col to the end
columns = [col for col in one_hot_encoded.columns if 'Unknown' not in col]
columns.append([col for col in one_hot_encoded.columns if 'Unknown' in col][0])
one_hot_encoded = one_hot_encoded[columns]

for col in reversed(one_hot_encoded.columns):
    merged_df.insert(merged_df.columns.get_loc('Roadway Type') + 1, col, one_hot_encoded[col])

merged_df.drop(columns= 'Roadway Type', inplace= True)


In [1597]:
merged_df['Roadway Surface'] = merged_df['Roadway Surface'].replace({
    'Other, see Narrative' : 'Other'
})

one_hot_encoded = pd.get_dummies(merged_df['Roadway Surface'], prefix='Roadway Surface')

for col in reversed(one_hot_encoded.columns):
    merged_df.insert(merged_df.columns.get_loc('Roadway Surface') + 1, col, one_hot_encoded[col])

merged_df.drop(columns= 'Roadway Surface', inplace= True)

In [1598]:
merged_df['Roadway Description'] = merged_df['Roadway Description'].replace({
    'Other, see Narrative' : 'Other, Unknown or Missing',
    'Missing / Degraded Markings' : 'Other, Unknown or Missing',
    'Unknown' : 'Other, Unknown or Missing'

})

one_hot_encoded = pd.get_dummies(merged_df['Roadway Description'], prefix='Roadway Description')

# put 'Other, Unknown or Missing' col to the end
columns = [col for col in one_hot_encoded.columns if 'Other, Unknown or Missing' not in col]
columns.append([col for col in one_hot_encoded.columns if 'Other, Unknown or Missing' in col][0])
one_hot_encoded = one_hot_encoded[columns]

for col in reversed(one_hot_encoded.columns):
    merged_df.insert(merged_df.columns.get_loc('Roadway Description') + 1, col, one_hot_encoded[col])

merged_df.drop(columns= 'Roadway Description', inplace= True)

In [1599]:
merged_df['Posted Speed Limit - Unknown'] = merged_df['Posted Speed Limit - Unknown'].apply(lambda x: 1 if x == 'Y' else 0)

In [1600]:
merged_df['Lighting'] = merged_df['Lighting'].replace({
    'Other, see Narrative' : 'Other or Unknown',
    'Unknown' : 'Other or Unknown'
})

one_hot_encoded = pd.get_dummies(merged_df['Lighting'], prefix='Lighting')

# put 'unknown' col to the end
columns = [col for col in one_hot_encoded.columns if 'Lighting_Dark - Unknown Lighting' not in col]
columns.append([col for col in one_hot_encoded.columns if 'Lighting_Dark - Unknown Lighting' in col][0])
one_hot_encoded = one_hot_encoded[columns]

for col in reversed(one_hot_encoded.columns):
    merged_df.insert(merged_df.columns.get_loc('Lighting') + 1, col, one_hot_encoded[col])

merged_df.drop(columns= 'Lighting', inplace= True)

In [1601]:
merged_df['Weather - Clear'] = merged_df['Weather - Clear'].apply(lambda x: 1 if x == 'Y' else 0)
merged_df['Weather - Snow'] = merged_df['Weather - Snow'].apply(lambda x: 1 if x == 'Y' else 0)
merged_df['Weather - Cloudy'] = merged_df['Weather - Cloudy'].apply(lambda x: 1 if x == 'Y' else 0)
merged_df['Weather - Fog/Smoke'] = merged_df['Weather - Fog/Smoke'].apply(lambda x: 1 if x == 'Y' else 0)
merged_df['Weather - Rain'] = merged_df['Weather - Rain'].apply(lambda x: 1 if x == 'Y' else 0)
merged_df['Weather - Severe Wind'] = merged_df['Weather - Severe Wind'].apply(lambda x: 1 if x == 'Y' else 0)
merged_df['Weather - Unknown'] = merged_df['Weather - Unknown'].apply(lambda x: 1 if x == 'Y' else 0)
merged_df['Weather - Other'] = merged_df['Weather - Other'].apply(lambda x: 1 if x == 'Y' else 0)

In [1602]:
one_hot_encoded = pd.get_dummies(merged_df['Highest Injury Severity Alleged'], prefix='Highest Injury Severity Alleged')

# put 'unknown' col to the end
columns = [col for col in one_hot_encoded.columns if 'Unknown' not in col]
columns.append([col for col in one_hot_encoded.columns if 'Unknown' in col][0])
one_hot_encoded = one_hot_encoded[columns]

for col in reversed(one_hot_encoded.columns):
    merged_df.insert(merged_df.columns.get_loc('Highest Injury Severity Alleged') + 1, col, one_hot_encoded[col])

merged_df.drop(columns= 'Highest Injury Severity Alleged', inplace= True)

In [1603]:
merged_df['Property Damage?'] = merged_df['Property Damage?'].apply(lambda x: 1 if x == 'Yes' else 0)
merged_df['CP Any Air Bags Deployed?'] = merged_df['CP Any Air Bags Deployed?'].apply(lambda x: 1 if x == 'Yes' else 0)
merged_df['CP Was Vehicle Towed?'] = merged_df['CP Was Vehicle Towed?'].apply(lambda x: 1 if x == 'Yes' else 0)
merged_df['CP Contact Area - Rear Left'] = merged_df['CP Contact Area - Rear Left'].apply(lambda x: 1 if x == 'Y' else 0)
merged_df['CP Contact Area - Left'] = merged_df['CP Contact Area - Left'].apply(lambda x: 1 if x == 'Y' else 0)
merged_df['CP Contact Area - Front Left'] = merged_df['CP Contact Area - Front Left'].apply(lambda x: 1 if x == 'Y' else 0)
merged_df['CP Contact Area - Rear'] = merged_df['CP Contact Area - Rear'].apply(lambda x: 1 if x == 'Y' else 0)
merged_df['CP Contact Area - Top'] = merged_df['CP Contact Area - Top'].apply(lambda x: 1 if x == 'Y' else 0)
merged_df['CP Contact Area - Front'] = merged_df['CP Contact Area - Front'].apply(lambda x: 1 if x == 'Y' else 0)
merged_df['CP Contact Area - Rear Right'] = merged_df['CP Contact Area - Rear Right'].apply(lambda x: 1 if x == 'Y' else 0)
merged_df['CP Contact Area - Right'] = merged_df['CP Contact Area - Right'].apply(lambda x: 1 if x == 'Y' else 0)
merged_df['CP Contact Area - Front Right'] = merged_df['CP Contact Area - Front Right'].apply(lambda x: 1 if x == 'Y' else 0)
merged_df['CP Contact Area - Bottom'] = merged_df['CP Contact Area - Bottom'].apply(lambda x: 1 if x == 'Y' else 0)
merged_df['CP Contact Area - Unknown'] = merged_df['CP Contact Area - Unknown'].apply(lambda x: 1 if x == 'Y' else 0)
merged_df['SV Any Air Bags Deployed?'] = merged_df['SV Any Air Bags Deployed?'].apply(lambda x: 1 if x == 'Yes' else 0)
merged_df['SV Was Vehicle Towed?'] = merged_df['SV Was Vehicle Towed?'].apply(lambda x: 1 if x == 'Yes' else 0)
merged_df['SV Were All Passengers Belted?'] = merged_df['SV Were All Passengers Belted?'].apply(lambda x: 1 if x == 'Yes' else 0)
merged_df['SV Pre-crash Speed - Unknown'] = merged_df['SV Pre-crash Speed - Unknown'].apply(lambda x: 1 if x == 'Y' else 0)
merged_df['SV Contact Area - Rear Left'] = merged_df['SV Contact Area - Rear Left'].apply(lambda x: 1 if x == 'Y' else 0)
merged_df['SV Contact Area - Left'] = merged_df['SV Contact Area - Left'].apply(lambda x: 1 if x == 'Y' else 0)
merged_df['SV Contact Area - Front Left'] = merged_df['SV Contact Area - Front Left'].apply(lambda x: 1 if x == 'Y' else 0)
merged_df['SV Contact Area - Rear'] = merged_df['SV Contact Area - Rear'].apply(lambda x: 1 if x == 'Y' else 0)
merged_df['SV Contact Area - Top'] = merged_df['SV Contact Area - Top'].apply(lambda x: 1 if x == 'Y' else 0)
merged_df['SV Contact Area - Front'] = merged_df['SV Contact Area - Front'].apply(lambda x: 1 if x == 'Y' else 0)
merged_df['SV Contact Area - Rear Right'] = merged_df['SV Contact Area - Rear Right'].apply(lambda x: 1 if x == 'Y' else 0)
merged_df['SV Contact Area - Right'] = merged_df['SV Contact Area - Right'].apply(lambda x: 1 if x == 'Y' else 0)
merged_df['SV Contact Area - Front Right'] = merged_df['SV Contact Area - Front Right'].apply(lambda x: 1 if x == 'Y' else 0)
merged_df['SV Contact Area - Bottom'] = merged_df['SV Contact Area - Bottom'].apply(lambda x: 1 if x == 'Y' else 0)
merged_df['SV Contact Area - Unknown'] = merged_df['SV Contact Area - Unknown'].apply(lambda x: 1 if x == 'Y' else 0)
merged_df['Data Availability - EDR'] = merged_df['Data Availability - EDR'].apply(lambda x: 1 if x == 'Y' else 0)
merged_df['Data Availability - Police Rpt'] = merged_df['Data Availability - Police Rpt'].apply(lambda x: 1 if x == 'Y' else 0)
merged_df['Data Availability - Telematics'] = merged_df['Data Availability - Telematics'].apply(lambda x: 1 if x == 'Y' else 0)
merged_df['Data Availability - Complaints'] = merged_df['Data Availability - Complaints'].apply(lambda x: 1 if x == 'Y' else 0)
merged_df['Data Availability - Video'] = merged_df['Data Availability - Video'].apply(lambda x: 1 if x == 'Y' else 0)
merged_df['Data Availability - Other'] = merged_df['Data Availability - Other'].apply(lambda x: 1 if x == 'Y' else 0)

merged_df.drop(columns= 'Data Availability - No Data', inplace= True)

merged_df['Data Availability - Unknown'] = merged_df['Data Availability - Unknown'].apply(lambda x: 1 if x == 'Y' else 0)


In [1604]:
print(merged_df['Law Enforcement Investigating?'].value_counts())



No         468
Yes        203
Unknown     28
Name: Law Enforcement Investigating?, dtype: int64


In [1605]:
merged_df.to_csv("NHTSA_ADS_cleaned.csv", index=False)